61人训练：采用集成学习
1，61人所有的数据，每次随机抽取50%，做三次，三次预测值取平均。average MAE:  19.249


In [1]:
import math
import torch
import gpytorch
import numpy as np
import pandas as pd
import gc
from sklearn.metrics import mean_absolute_error
from LBFGS import FullBatchLBFGS
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
torch.set_default_tensor_type(torch.DoubleTensor)

data=pd.read_csv('QingData_sbp.csv')
#data = pd.read_csv('data_20p_1h_21f.csv')
#data=data.sample(frac=0.5,random_state=1)
patient_list=data['patient_id'].unique()
#print(patient_list) 
####################################SBP 加‘20’
feature=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20']
#feature=['HrRR','HrSS','AmFS','AmSN','AmFN','AmFN_FS','AmFN_SN','TmFN','TmNF','TmFN_NF','Tm_FS','Tm_SF','Tm_SN','Tm_FQ','PAT_S','PAT_F','PAT_Q','ArFS','ArSN','ArNF','ArNF_FN']
n_devices = 3#torch.cuda.device_count()
print('Planning to run on {} GPUs.'.format(n_devices))

Planning to run on 3 GPUs.


In [2]:
MAE=0
ME=0
STD=0
L=len(patient_list)
prediction=[]
for id in range(L):
    scaler=StandardScaler()
    output_device = torch.device('cuda:0')
    
    source_data=data[data['patient_id']!=patient_list[id]]
    target_data=data[data['patient_id']==patient_list[id]]
    source_x=source_data[feature]
    source_x=scaler.fit_transform(source_x)
    source_y=source_data['SBP']
    
    test_x=target_data[feature]
    test_x=scaler.transform(test_x)
    test_y=target_data['SBP']
    test_x=torch.from_numpy( test_x )
    test_y=torch.from_numpy( np.array(test_y))
    test_x, test_y = test_x.to(output_device), test_y.to(output_device)
    
    
    def predict(random):
        train_x, _, train_y, _=train_test_split(source_x, source_y, test_size=0.5,random_state=random)
        train_x=torch.from_numpy( train_x )
        train_y=torch.from_numpy( np.array(train_y) )
        train_x, train_y = train_x.to(output_device), train_y.to(output_device)
        checkpoint_size = find_best_gpu_setting(train_x, train_y,
                                        n_devices=n_devices,
                                        output_device=output_device,
                                        preconditioner_size=100)
        
        model, likelihood = train(train_x, train_y,
                                  n_devices=n_devices, output_device=output_device,
                                  checkpoint_size=checkpoint_size,
                                  preconditioner_size=100,
                                  n_training_iter=100)
        
        model.eval()
        likelihood.eval()
        with torch.no_grad(),\
             gpytorch.settings.fast_pred_var(),\
             gpytorch.beta_features.checkpoint_kernel(checkpoint_size):        
            y_preds = likelihood(model(test_x))
            y_mean=y_preds.mean
        
        return y_mean.cpu()
        
    

    class ExactGPModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood, n_devices):
            super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = gpytorch.means.ConstantMean()
            base_covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=0.5))

            self.covar_module = gpytorch.kernels.MultiDeviceKernel(
                base_covar_module, device_ids=range(n_devices),
                output_device=output_device
            )

        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    def train(train_x,
              train_y,
              n_devices,
              output_device,
              checkpoint_size,
              preconditioner_size,
              n_training_iter,
    ):
        likelihood = gpytorch.likelihoods.GaussianLikelihood().to(output_device)
        model = ExactGPModel(train_x, train_y, likelihood, n_devices).to(output_device)
        model.train()
        likelihood.train()

        optimizer = FullBatchLBFGS(model.parameters(), lr=0.1)
        # "Loss" for GPs - the marginal log likelihood
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)


        with gpytorch.beta_features.checkpoint_kernel(checkpoint_size), \
             gpytorch.settings.max_preconditioner_size(preconditioner_size):

            def closure():
                optimizer.zero_grad()
                output = model(train_x)
                loss = -mll(output, train_y)
                return loss

            loss = closure()
            loss.backward()

            for i in range(n_training_iter):
                options = {'closure': closure, 'current_loss': loss, 'max_ls': 10}
                loss, _, _, _, _, _, _, fail = optimizer.step(options)

#                 print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
#                     i + 1, n_training_iter, loss.item(),
#                     model.covar_module.module.base_kernel.lengthscale.item(),
#                     model.likelihood.noise.item()
#                 ))

                if fail:
                    print('Convergence reached!')
                    break

        print(f"Finished training on {train_x.size(0)} data points using {n_devices} GPUs.")
        return model, likelihood
    
    def find_best_gpu_setting(train_x,
                              train_y,
                              n_devices,
                              output_device,
                              preconditioner_size
    ):
        N = train_x.size(0)

        # Find the optimum partition/checkpoint size by decreasing in powers of 2
        # Start with no partitioning (size = 0)
        settings = [0] + [int(n) for n in np.ceil(N / 2**np.arange(1, np.floor(np.log2(N))))]

        for checkpoint_size in settings:
            print('Number of devices: {} -- Kernel partition size: {}'.format(n_devices, checkpoint_size))
            try:
                # Try a full forward and backward pass with this setting to check memory usage
                _, _ = train(train_x, train_y,
                             n_devices=n_devices, output_device=output_device,
                             checkpoint_size=checkpoint_size,
                             preconditioner_size=preconditioner_size, n_training_iter=1)

                # when successful, break out of for-loop and jump to finally block
                break
            except RuntimeError as e:
                print('RuntimeError: {}'.format(e))
            except AttributeError as e:
                print('AttributeError: {}'.format(e))
            finally:
                # handle CUDA OOM error
                gc.collect()
                torch.cuda.empty_cache()
        return checkpoint_size

    # Set a large enough preconditioner size to reduce the number of CG iterations run

    prediction=(predict(1)+predict(2)+predict(3))/3
    mae=mean_absolute_error(test_y.cpu(),prediction)
    MAE+=mae

          
    error=(test_y.cpu()-prediction).numpy()
    me=np.mean(error)
    ME+=me
    std=np.std(error,ddof=1)
    STD+=std*(test_x.cpu().shape[0]-1)          
    print(patient_list[id],'MAE: ', format(mae,'.3f'),'ME',format(me,'.3f'),'STD: ',format(std,'.3f'))


#print('average MAE: ', format(MAE/L,'.3f'))
print('average MAE: ', format(MAE/L,'.3f'),'ME: ', format(ME/L,'.3f'), 'STD: ', format(STD/(data.shape[0]-L),'.3f'))

Number of devices: 3 -- Kernel partition size: 0


/home/staff_qing_liu/Jiacheng /LBFGS.py:257: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554788289/work/torch/csrc/utils/python_arg_parser.cpp:1005.)
  p.data.add_(step_size, update[offset:offset + numel].view_as(p.data))


Finished training on 4978 data points using 3 GPUs.
Convergence reached!
Finished training on 4978 data points using 3 GPUs.
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4978 data points using 3 GPUs.
Convergence reached!
Finished training on 4978 data points using 3 GPUs.
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4978 data points using 3 GPUs.
Convergence reached!
Finished training on 4978 data points using 3 GPUs.
1 MAE:  34.156 ME -34.141 STD:  13.750
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4942 data points using 3 GPUs.
Convergence reached!
Finished training on 4942 data points using 3 GPUs.
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4942 data points using 3 GPUs.
Convergence reached!
Finished training on 4942 data points using 3 GPUs.
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4942 data points using 3 GPUs.
Convergence reached!
Finished training 

Finished training on 4996 data points using 3 GPUs.
Convergence reached!
Finished training on 4996 data points using 3 GPUs.
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4996 data points using 3 GPUs.
Convergence reached!
Finished training on 4996 data points using 3 GPUs.
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4996 data points using 3 GPUs.
Convergence reached!
Finished training on 4996 data points using 3 GPUs.
20 MAE:  15.303 ME 14.216 STD:  11.839
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4956 data points using 3 GPUs.
Convergence reached!
Finished training on 4956 data points using 3 GPUs.
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4956 data points using 3 GPUs.
Convergence reached!
Finished training on 4956 data points using 3 GPUs.
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4956 data points using 3 GPUs.
Convergence reached!
Finished training 

Finished training on 5045 data points using 3 GPUs.
Convergence reached!
Finished training on 5045 data points using 3 GPUs.
34 MAE:  9.038 ME 0.034 STD:  11.045
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4989 data points using 3 GPUs.
Convergence reached!
Finished training on 4989 data points using 3 GPUs.
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4989 data points using 3 GPUs.
Convergence reached!
Finished training on 4989 data points using 3 GPUs.
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4989 data points using 3 GPUs.
Convergence reached!
Finished training on 4989 data points using 3 GPUs.
35 MAE:  14.344 ME -12.910 STD:  14.660
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4977 data points using 3 GPUs.
Convergence reached!
Finished training on 4977 data points using 3 GPUs.
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4977 data points using 3 GPUs.
C

Finished training on 4987 data points using 3 GPUs.
Convergence reached!
Finished training on 4987 data points using 3 GPUs.
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4987 data points using 3 GPUs.
Convergence reached!
Finished training on 4987 data points using 3 GPUs.
49 MAE:  13.676 ME -1.519 STD:  15.837
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4975 data points using 3 GPUs.
Convergence reached!
Finished training on 4975 data points using 3 GPUs.
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4975 data points using 3 GPUs.
Convergence reached!
Finished training on 4975 data points using 3 GPUs.
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4975 data points using 3 GPUs.
Convergence reached!
Finished training on 4975 data points using 3 GPUs.
50 MAE:  12.912 ME -9.621 STD:  12.295
Number of devices: 3 -- Kernel partition size: 0
Finished training on 4976 data points using 3 GPUs.
